# **Preparation**

- Edit > Notebook settings > Hardward accelerators > GPU > SAVE
- Download the Friends dataset in EmotionLines website:
http://doraemon.iis.sinica.edu.tw/emotionlines/download.html
- Download the unlabeled json file.

##### **Settings**

In [ ]:
!pip install transformers --quiet # package installer for python

     |████████████████████████████████| 1.5MB 9.0MB/s 
     |████████████████████████████████| 2.9MB 30.0MB/s 
     |████████████████████████████████| 890kB 60.6MB/s 


In [ ]:
import torch
from transformers import BertModel, BertTokenizer

In [ ]:
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

# **Emotion Recognition**

##### **Dataset**

In [ ]:
import json

emotion_cnt = {}

data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
  for dialog in json.loads(open('friends_' + dtype + '.json').read()):
    for line in dialog:
      data[dtype]['speaker'].append(line['speaker'])
      data[dtype]['utterance'].append(line['utterance'])
      data[dtype]['emotion'].append(line['emotion'])
      if( line['emotion'] in emotion_cnt ):
        emotion_cnt[line['emotion']] = emotion_cnt[line['emotion']] + 1
      else:
        emotion_cnt[line['emotion']] = 1

In [ ]:
emotion_cnt

{'anger': 759,
 'disgust': 331,
 'fear': 246,
 'joy': 1710,
 'neutral': 6530,
 'non-neutral': 2772,
 'sadness': 498,
 'surprise': 1657}

In [ ]:
e2i_dict = dict((emo, i) for i, emo in enumerate(set(data['train']['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}

In [ ]:
#Weights Calculation 로직 추가
weights = []
samples_n = len(data['train']['emotion'])
class_n = len(i2e_dict)
print(samples_n)
print(class_n)

for c in i2e_dict:
  emotion_label = i2e_dict[c]
  samples_n_c = emotion_cnt[emotion_label]
  w = round( samples_n / (class_n*samples_n_c) , 2)
  print(emotion_label+':'+str(samples_n_c)+','+str(w))
  weights.append(w)

print(weights)


10561
8
disgust:331,3.99
joy:1710,0.77
surprise:1657,0.8
anger:759,1.74
sadness:498,2.65
fear:246,5.37
neutral:6530,0.2
non-neutral:2772,0.48
[3.99, 0.77, 0.8, 1.74, 2.65, 5.37, 0.2, 0.48]


##### **Model**

In [ ]:
import torch.nn as nn
from transformers import BertModel, BertTokenizer, BertConfig

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
    self.bert_config = BertConfig(attention_probs_dropout_prob=0.5, hidden_dropout_prob=0.5) #Dropout 0.5로 변경
    self.bert_model = BertModel.from_pretrained(pretrained_weights)
    self.linear = torch.nn.Linear(768, len(e2i_dict))

  def forward(self, utterance, speaker):
    tokens = self.bert_tokenizer.tokenize(utterance)
    
    #두문장인 경우 [SEP] 토큰 추가
    new_tokens = []
    for token in tokens:
      new_tokens = new_tokens + [token]
      if(token=='.' or token=='?'):
        new_tokens = new_tokens + ['[SEP]']

    if(new_tokens[-1]!='[SEP]'):
      new_tokens = new_tokens + ['[SEP]']

    tokens = ['[CLS]'] + new_tokens
    ids = [tokenizer.convert_tokens_to_ids(tokens)] # (bat=1, len)
    input_tensor = torch.tensor(ids).cuda()

    hidden_tensor = self.bert_model(input_tensor)[0] # (bat, len, hid)
    hidden_tensor = hidden_tensor[:, 0, :] # (bat, hid)
    logit = self.linear(hidden_tensor)
    return logit

##### **Evaluation Metrics**

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(true_list, pred_list):
  precision = precision_score(true_list, pred_list, average=None)
  recall = recall_score(true_list, pred_list, average=None)
  micro_f1 = f1_score(true_list, pred_list, average='micro')
  print('precision:\t', ['%.4f' % v for v in precision])
  print('recall:\t\t', ['%.4f' % v for v in recall])
  print('micro_f1: %.6f' % micro_f1)

##### **Hyper-parameters**

In [ ]:
pretrained_weights = 'bert-base-uncased'
learning_rate = 1e-5
n_epoch = 3
batch_size = 1

##### **Training**

In [ ]:
import os
import math
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from tqdm import tqdm_notebook

model = Model()
model.cuda()
class_weights = torch.FloatTensor(weights).cuda()
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)#
criterion = torch.nn.CrossEntropyLoss() # LogSoftmax & NLLLoss
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

for i_epoch in range(n_epoch):
  print('i_epoch:', i_epoch)

  model.train()
  for i_batch in tqdm_notebook(range(len(data['train']['utterance']))):
    logit = model(data['train']['utterance'][i_batch], data['train']['speaker'][i_batch])
    target = torch.tensor([e2i_dict[data['train']['emotion'][i_batch]]]).cuda()
    loss = criterion(logit, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  
  model.eval()
  pred_list, true_list = [], []
  for i_batch in tqdm_notebook(range(len(data['dev']['utterance']))):
    logit = model(data['dev']['utterance'][i_batch], data['dev']['speaker'][i_batch])
    _, max_idx = torch.max(logit, dim=-1)
    pred_list += max_idx.tolist()
    true_list += [e2i_dict[data['dev']['emotion'][i_batch]]]
  evaluate(pred_list, true_list) # print results

i_epoch: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



precision:	 ['0.0000', '0.6260', '0.5695', '0.2706', '0.0968', '0.0000', '0.8961', '0.1776']
recall:		 ['0.0000', '0.5580', '0.5584', '0.3770', '0.7500', '0.0000', '0.6587', '0.2550']
micro_f1: 0.568761
i_epoch: 1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



precision:	 ['0.0000', '0.7073', '0.5828', '0.3294', '0.2581', '0.0000', '0.8900', '0.1963']
recall:		 ['0.0000', '0.5437', '0.6027', '0.4179', '0.4848', '0.0000', '0.6970', '0.2897']
micro_f1: 0.592530
i_epoch: 2



precision:	 ['0.0000', '0.6911', '0.6159', '0.3529', '0.3065', '0.0000', '0.8717', '0.2336']
recall:		 ['0.0000', '0.5822', '0.6078', '0.4615', '0.5135', '0.0000', '0.6971', '0.3086']
micro_f1: 0.598472


In [ ]:
data['train']['speaker']

##### **Labeling**


In [ ]:
from collections import OrderedDict

labeled = []
dialogs = json.loads(open('unlabeled.json').read())
for dialog in tqdm_notebook(dialogs):
  dialog_list = []
  for line in dialog:
    logit = model(line['utterance'])
    _, max_idx = torch.max(logit, dim=-1)
    pred_emotion = max_idx.tolist()[0]

    line_dict = OrderedDict()
    line_dict['speaker'] = line['speaker']
    line_dict['utterance'] = line['utterance']
    line_dict['emotion'] = i2e_dict[pred_emotion]
    dialog_list.append(line_dict)
  labeled.append(dialog_list)

with open('labeled.json', 'w') as json_file:
  json.dump(labeled, json_file, indent='\t')

In [ ]:
import csv

dialogs = []
dialogs.append([])

with open('en_data.csv', newline='') as csvfile:
  spamreader = csv.reader(csvfile)
  for i,row in enumerate(spamreader):
    if i!=0:
      dialogs[0].append({'id':row[0],'speaker':row[3], 'utterance':row[4]})

from collections import OrderedDict

labeled = []
#dialogs = json.loads(open('unlabeled.json').read())
for dialog in tqdm_notebook(dialogs):
  dialog_list = []
  for line in dialog:
    logit = model(line['utterance'], line['speaker'])
    _, max_idx = torch.max(logit, dim=-1)
    pred_emotion = max_idx.tolist()[0]

    line_dict = OrderedDict()
    line_dict['Id'] = line['id']
    line_dict['speaker'] = line['speaker']
    line_dict['utterance'] = line['utterance']
    line_dict['emotion'] = i2e_dict[pred_emotion]
    dialog_list.append(line_dict)
  labeled.append(dialog_list)

with open('labeled.csv', 'w', newline='') as csvfile:
    fieldnames = ['Id', 'Predicted']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in dialog_list:
      writer.writerow({'Id': row['Id'], 'Predicted': row['emotion']})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


##### **Proposal**

- There is a class imbalance problem. (Use weighted cross-entropy etc.)

- Our model takes a single sentence. (Make it grasp its context as well.)

- Our model does not consider speaker information. (Make it consider the info.)

- Batch size is set as 1. (Increase the batch size.)